In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

from tensorflow.keras.optimizers import Adam

2022-08-03 13:12:01.375262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 13:12:01.375281: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
image_dir = Path('output/drone_dataset_28072022011417/images')
data_path_1 = Path('output/drone_dataset_28072022011417/data/drone.csv')
data_path_2 = Path('output/drone_dataset_29072022105203/data/drone.csv')
data_path_3 = Path('output/drone_dataset_29072022120237/data/drone.csv')

# Create File DataFrame

In [4]:
image_df_1 = pd.read_csv(data_path_1, usecols = ["img_path", "dist"]);
image_df_2 = pd.read_csv(data_path_2, usecols = ["img_path", "dist"]);
image_df_3 = pd.read_csv(data_path_3, usecols = ["img_path", "dist"]);
image_df = pd.concat([image_df_1, image_df_2, image_df_3], axis=0)

In [8]:
train_df, test_df = train_test_split(image_df_1, train_size=0.8, shuffle=False)

# Loading Images

In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [12]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='img_path',
    y_col='dist',
    target_size=(512, 512),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='img_path',
    y_col='dist',
    target_size=(512, 512),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='img_path',
    y_col='dist',
    target_size=(512, 512),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 640 validated image filenames.
Found 160 validated image filenames.
Found 200 validated image filenames.


# Training

In [13]:
inputs = tf.keras.Input(shape=(512, 512, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss="mse"
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

2022-08-03 13:12:11.619038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 13:12:11.619239: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 13:12:11.619271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-03 13:12:11.619299: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-03 13:12:11.620581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/100
20/20 [==============================] - 41s 2s/step - loss: 280.0924 - val_loss: 47.1703
Epoch 2/100
20/20 [==============================] - 40s 2s/step - loss: 46.5158 - val_loss: 32.5411
Epoch 3/100
20/20 [==============================] - 40s 2s/step - loss: 34.0736 - val_loss: 35.4208
Epoch 4/100
20/20 [==============================] - 39s 2s/step - loss: 33.8146 - val_loss: 31.6337
Epoch 5/100
20/20 [==============================] - 39s 2s/step - loss: 34.4504 - val_loss: 34.5110
Epoch 6/100
20/20 [==============================] - 39s 2s/step - loss: 34.3098 - val_loss: 37.8315
Epoch 7/100
20/20 [==============================] - 39s 2s/step - loss: 36.3121 - val_loss: 31.2281
Epoch 8/100
20/20 [==============================] - 39s 2s/step - loss: 36.1272 - val_loss: 32.3865
Epoch 9/100
20/20 [==============================] - 39s 2s/step - loss: 33.7381 - val_loss: 31.1683
Epoch 10/100
20/20 [==============================] - 39s 2s/step - loss: 34.1927 - val_lo

# Results

In [16]:
predicted_dist = np.squeeze(model.predict(test_images))
true_dist = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_dist, predicted_dist)
print("Test R^2 Score: {:.5f}".format(r2))

7/7 [==============================] - 2s 312ms/step
     Test RMSE: 5.69711
Test R^2 Score: -0.16677


# Comparison

In [20]:
np.stack((predicted_dist, true_dist), axis=-1)

array([[22.02230644, 15.5241747 ],
       [21.82486343, 20.63976744],
       [21.76651955, 25.01999201],
       [21.72676086, 27.14774392],
       [21.78040123, 24.12467616],
       [21.95899963, 15.06651917],
       [21.66426468, 26.43860813],
       [21.77472687, 24.18677324],
       [22.49593163,  8.83176087],
       [21.81474113, 21.3541565 ],
       [21.9824276 , 14.59451952],
       [21.70908546, 13.19090596],
       [21.74826241, 24.41311123],
       [21.74182701, 26.41968963],
       [21.74633217, 25.01999201],
       [21.93522835, 16.09347694],
       [22.02865791, 14.45683229],
       [21.85119057, 22.91287847],
       [21.74867058, 24.16609195],
       [21.89743423, 19.02629759],
       [21.85177422, 20.42057786],
       [21.75837898, 24.10394159],
       [21.80548668, 20.85665361],
       [21.80816078, 23.10844002],
       [21.99903297, 16.673332  ],
       [21.81831169, 23.87467277],
       [21.78380775, 21.65640783],
       [21.77282524, 20.61552813],
       [21.81557655,

In [21]:
null_rmse = np.sqrt(np.sum((true_dist - np.mean(true_dist))**2) / len(true_dist))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 5.27428


# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/9AnCNBL8c6Q